In [1]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
KEY = os.getenv("TOGETHER_API_KEY")

In [6]:
print(KEY)

7ebf4022abdc280634e753f9db99f15a862a70e78e02ca93ae22195aa3503803


In [ ]:
llm = ChatOpenAI(
    openai_api_key=KEY, 
    model_name="mistralai/Mistral-7B-Instruct-v0.2",  # or any Together model
    openai_api_base="https://api.together.xyz/v1",    # <-- required
    temperature=0.3
)
